### Task [102](https://github.com/salgo60/ProjectOutdoorGyms/issues/102) "uppdater Gävle utegym"
 se även [Notebook Stockholm](https://github.com/salgo60/ProjectOutdoorGyms/issues/99)

* [PSI data Gävle](https://www.gavle.se/kommunens-service/kommun-och-politik/statistik-fakta-och-oppna-data/oppna-data/datakatalog/)
  * [datasets utegym-1](https://www.gavle.se/kommunens-service/kommun-och-politik/statistik-fakta-och-oppna-data/oppna-data/datakatalog/data/#esc_entry=357&esc_context=1) --> ?? records compare [WD projekt Sweden](https://www.wikidata.org/wiki/Wikidata:WikiProject_Outdoor_Gyms#Sweden) --> 13 records
  * API call json
 
* [denna notebook](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/Gävle%20outdoor%20gym.ipynb)

* https://github.com/salgo60/ProjectOutdoorGyms

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-05-29 17:43:22.373108


In [29]:
url = 'https://catalog.gavle.se/store/1/resource/360'

from io import BytesIO
from zipfile import ZipFile
import urllib.request
from urllib.parse import urlparse
url = urllib.request.urlopen(url)

with ZipFile(BytesIO(url.read())) as my_zip_file:
    for contained_file in my_zip_file.namelist():
        # with open(("unzipped_and_read_" + contained_file + ".file"), "wb") as output:
        for line in my_zip_file.open(contained_file).readlines():
            print(type(line))
            data = line.decode('utf-8')
            dataJson = json.loads(data)
            #print(dataJson)

#Utegymdata = pd.DataFrame(dataJson["features"]) 
#Utegymdata.keys() 
Utegymdata= pd.json_normalize(dataJson["features"])
Utegymdata.keys()  
Utegymdata

<class 'bytes'>


,type,geometry.type,geometry.coordinates,properties.NAMN
0,Feature,Point,"[17.1124, 60.6727]",Stora Vall
1,Feature,Point,"[17.0948, 60.6782]",Lexevägen
2,Feature,Point,"[17.2343, 60.6731]",Rostadsvägen
3,Feature,Point,"[16.8966, 60.6151]",Pilparken
4,Feature,Point,"[17.1135, 60.6744]",Stadsbadet
5,Feature,Point,"[17.1684, 60.6794]",Toppsockergränd
6,Feature,Point,"[17.0991, 60.687]",Pingeltorp lekplats
7,Feature,Point,"[17.1587, 60.6429]",Skogslekplatsen Hemlingby
8,Feature,Point,"[16.8866, 60.6186, 0]",Forsbacka Utegym
9,Feature,Point,"[17.1707, 60.6494, 0]",Hemlingby Utegym


### Status JSON 
* see [notebook Stockholm](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/Stockholm%20outdoor%20gym.ipynb) med 100 utegym och bra flöde av nya....


In [30]:
Utegymdata.to_csv("Gävle_gym.csv")

### SPARQL Wikidata
Hämta utegym i WD för Jönköpings kommun [wd:Q510010](https://www.wikidata.org/wiki/Q510010?uselang=sv)
* [SPARQL](https://w.wiki/5DTs) Wikidata Gävle kommun

In [31]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# SPARQL https://w.wiki/5DTs
queryGävle = """SELECT distinct ?inception ?item ?itemLabel ?www ?id ?API  WHERE {
  ?item wdt:P6104 wd:Q107186275.
  OPTIONAL {?item wdt:P571 ?inception}
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "sv,en,de" . 
  }
  
OPTIONAL{?item wdt:P571 ?date}
OPTIONAL { ?item wdt:P18 ?img}
?item wdt:P131 wd:Q510010  # kommun
OPTIONAL { ?item wdt:P856 ?www. }
OPTIONAL { ?item wdt:P625 ?coord. }
OPTIONAL { ?item wdt:P217 ?id. }
} order by desc(?date)"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDtotOutdoorGym = get_sparql_dataframe(endpoint_url, queryGävle)

WDtotOutdoorGym


,inception,item,itemLabel,www,id,API
0,None,http://www.wikidata.org/entity/Q107209364,"Lexevägen, utegym",None,None,None
1,None,http://www.wikidata.org/entity/Q107211024,"Stora Vall, utegym",None,None,None
2,None,http://www.wikidata.org/entity/Q107211026,"Rostadsvägen, utegym",None,None,None
3,None,http://www.wikidata.org/entity/Q107211027,"Pilparken, utegym",None,None,None
4,None,http://www.wikidata.org/entity/Q107211028,"Stadsbadet, utegym",None,None,None
5,None,http://www.wikidata.org/entity/Q107211029,"Toppsockergränd, utegym",None,None,None
6,None,http://www.wikidata.org/entity/Q107211031,"Pingeltorp lekplats, utegym",None,None,None
7,None,http://www.wikidata.org/entity/Q107211032,"Skogslekplatsen Hemlingby, utegym",None,None,None
8,None,http://www.wikidata.org/entity/Q107211033,Forsbacka utegym,https://www.gavle.se/kommunens-service/kultur-...,None,None
9,None,http://www.wikidata.org/entity/Q107211034,"Hemlingby, utegym",https://www.gavle.se/kommunens-service/kultur-...,None,None


In [32]:
WDtotOutdoorGym.isnull().sum(), WDtotOutdoorGym.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   inception  0 non-null      object
 1   item       13 non-null     object
 2   itemLabel  13 non-null     object
 3   www        5 non-null      object
 4   id         0 non-null      object
 5   API        0 non-null      object
dtypes: object(6)
memory usage: 752.0+ bytes


(inception    13
 item          0
 itemLabel     0
 www           8
 id           13
 API          13
 dtype: int64,
 None)

In [33]:
Utegymdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   type                  12 non-null     object
 1   geometry.type         12 non-null     object
 2   geometry.coordinates  12 non-null     object
 3   properties.NAMN       12 non-null     object
dtypes: object(4)
memory usage: 512.0+ bytes


### Saknas unikt ID i Gäcle data --> fuling hoppas namnet like

In [35]:
WDGefleNew = WDtotOutdoorGym.rename(columns={'id': 'properties.NAMN'}) 
WDGefleNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   inception        0 non-null      object
 1   item             13 non-null     object
 2   itemLabel        13 non-null     object
 3   www              5 non-null      object
 4   properties.NAMN  0 non-null      object
 5   API              0 non-null      object
dtypes: object(6)
memory usage: 752.0+ bytes


In [36]:
OutdoorGym_diff = pd.merge(WDUGefleNew,Utegymdata, how='outer', indicator='Exist')

/Users/magnus/Library/Python/3.9/lib/python/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [37]:
OutdoorGym_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   inception             0 non-null      object  
 1   item                  13 non-null     object  
 2   itemLabel             13 non-null     object  
 3   www                   5 non-null      object  
 4   properties.NAMN       12 non-null     object  
 5   API                   0 non-null      object  
 6   type                  12 non-null     object  
 7   geometry.type         12 non-null     object  
 8   geometry.coordinates  12 non-null     object  
 9   Exist                 25 non-null     category
dtypes: category(1), object(9)
memory usage: 2.1+ KB


In [38]:
OutdoorGym_diff["Exist"].value_counts()

left_only     13
right_only    12
both           0
Name: Exist, dtype: int64

In [40]:
#where Wikidata miss an ID in OutdoorGym_diff
MissingGymsWD = OutdoorGym_diff[OutdoorGym_diff["Exist"] == "right_only"]

### Saknas i wikidata
Blir inte bra utan unikt ID

In [41]:
MissingGymsWD

,inception,item,itemLabel,www,properties.NAMN,API,type,geometry.type,geometry.coordinates,Exist
13,NaN,NaN,NaN,NaN,Stora Vall,NaN,Feature,Point,"[17.1124, 60.6727]",right_only
14,NaN,NaN,NaN,NaN,Lexevägen,NaN,Feature,Point,"[17.0948, 60.6782]",right_only
15,NaN,NaN,NaN,NaN,Rostadsvägen,NaN,Feature,Point,"[17.2343, 60.6731]",right_only
16,NaN,NaN,NaN,NaN,Pilparken,NaN,Feature,Point,"[16.8966, 60.6151]",right_only
17,NaN,NaN,NaN,NaN,Stadsbadet,NaN,Feature,Point,"[17.1135, 60.6744]",right_only
18,NaN,NaN,NaN,NaN,Toppsockergränd,NaN,Feature,Point,"[17.1684, 60.6794]",right_only
19,NaN,NaN,NaN,NaN,Pingeltorp lekplats,NaN,Feature,Point,"[17.0991, 60.687]",right_only
20,NaN,NaN,NaN,NaN,Skogslekplatsen Hemlingby,NaN,Feature,Point,"[17.1587, 60.6429]",right_only
21,NaN,NaN,NaN,NaN,Forsbacka Utegym,NaN,Feature,Point,"[16.8866, 60.6186, 0]",right_only
22,NaN,NaN,NaN,NaN,Hemlingby Utegym,NaN,Feature,Point,"[17.1707, 60.6494, 0]",right_only


In [42]:
MissingGymsKommun = OutdoorGym_diff[OutdoorGym_diff["Exist"] == "left_only"]
MissingGymsKommun

,inception,item,itemLabel,www,properties.NAMN,API,type,geometry.type,geometry.coordinates,Exist
0,None,http://www.wikidata.org/entity/Q107209364,"Lexevägen, utegym",None,NaN,None,NaN,NaN,NaN,left_only
1,None,http://www.wikidata.org/entity/Q107211024,"Stora Vall, utegym",None,NaN,None,NaN,NaN,NaN,left_only
2,None,http://www.wikidata.org/entity/Q107211026,"Rostadsvägen, utegym",None,NaN,None,NaN,NaN,NaN,left_only
3,None,http://www.wikidata.org/entity/Q107211027,"Pilparken, utegym",None,NaN,None,NaN,NaN,NaN,left_only
4,None,http://www.wikidata.org/entity/Q107211028,"Stadsbadet, utegym",None,NaN,None,NaN,NaN,NaN,left_only
5,None,http://www.wikidata.org/entity/Q107211029,"Toppsockergränd, utegym",None,NaN,None,NaN,NaN,NaN,left_only
6,None,http://www.wikidata.org/entity/Q107211031,"Pingeltorp lekplats, utegym",None,NaN,None,NaN,NaN,NaN,left_only
7,None,http://www.wikidata.org/entity/Q107211032,"Skogslekplatsen Hemlingby, utegym",None,NaN,None,NaN,NaN,NaN,left_only
8,None,http://www.wikidata.org/entity/Q107211033,Forsbacka utegym,https://www.gavle.se/kommunens-service/kultur-...,NaN,None,NaN,NaN,NaN,left_only
9,None,http://www.wikidata.org/entity/Q107211034,"Hemlingby, utegym",https://www.gavle.se/kommunens-service/kultur-...,NaN,None,NaN,NaN,NaN,left_only


In [43]:
# Brukar vara en "dum" tanke att kommunen bara visar sina utegym inte andras som är gratis...
# Med brasklappen att Wikidata kan alla redigera och datat kan vara fel
MissingGymsKommun[["item","itemLabel","www"]]

,item,itemLabel,www
0,http://www.wikidata.org/entity/Q107209364,"Lexevägen, utegym",None
1,http://www.wikidata.org/entity/Q107211024,"Stora Vall, utegym",None
2,http://www.wikidata.org/entity/Q107211026,"Rostadsvägen, utegym",None
3,http://www.wikidata.org/entity/Q107211027,"Pilparken, utegym",None
4,http://www.wikidata.org/entity/Q107211028,"Stadsbadet, utegym",None
5,http://www.wikidata.org/entity/Q107211029,"Toppsockergränd, utegym",None
6,http://www.wikidata.org/entity/Q107211031,"Pingeltorp lekplats, utegym",None
7,http://www.wikidata.org/entity/Q107211032,"Skogslekplatsen Hemlingby, utegym",None
8,http://www.wikidata.org/entity/Q107211033,Forsbacka utegym,https://www.gavle.se/kommunens-service/kultur-...
9,http://www.wikidata.org/entity/Q107211034,"Hemlingby, utegym",https://www.gavle.se/kommunens-service/kultur-...


In [18]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-05-29 16:36:43.213903
Time elapsed (hh:mm:ss.ms) 0:00:01.487016
